<a href="https://colab.research.google.com/github/Macklees928/me/blob/main/Llama_2_HODL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cooking with LLMs: LLaMa 2 Zero-Shot and Few-Shot


Primary Author: Hayden Ratliff (hratliff@mit.edu)

Teammates: Iris Brook, Joe Kajon, Mackenzie Lees

## Import packages & load data

In [ ]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path to the folder
hodl_project_path = '/content/drive/My Drive/Colab Notebooks/HODL Project' # hayden

# load in dataset from google drive
rawData = pd.read_csv(hodl_project_path + "/full_dataset.csv")

In [ ]:
rawData.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
# downloading subset of the data for custom GPT model
import ast

subset = rawData[["NER", "directions"]].loc[0:99,:]
subset["NER"] = subset["NER"].transform(lambda x: ", ".join(ast.literal_eval(x)))
subset["directions"] = subset["directions"].transform(lambda x: " ".join(ast.literal_eval(x)))

subset.rename(columns={"NER": "Ingredients", "directions": "Recipe"}, inplace=True)

subset.to_csv("formatted_subset.csv")

## Modeling

### Load Llama model

Adapted from: https://colab.research.google.com/drive/1SQmK0GYz34RGVlOnL5YMkdm7hXD6OjQT?usp=sharing#scrollTo=jsBrtGpZYmcQ

In [ ]:
!pip install transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.6 MB/s eta 0:00:00


In [ ]:
! git config --global credential.helper store

In [ ]:
! huggingface-cli login
# token: [your token here]
# NOTE: you need to generate a huggingface token and input it here at login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
import time

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### Instructions and Ingredients

In [ ]:
# 3 sets of instructions
instructions_1 = "Make me a recipe with these ingredients:"
instructions_2 = "Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”."
instructions_3 = """You are an expert chef who is highly skilled at providing cooking instructions. When given a list of ingredients, please:
1. Provide a step by step list of instructions for cooking a meal.
2. The instructions will only require the listed ingredients.
3. The instructions may choose not to use some of the listed ingredients.
4. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”"""

# 3 sets of ingredients
ingredients_1 = "Chicken, garlic, spinach, peppers, olive oil, salt, pepper"
ingredients_2 = "Flour, sugar, butter, eggs, milk, chocolate chips, oil, whipped cream, frosting"
ingredients_3 = "Broccoli, strawberries, pineapple, JELLO mix, raisins, pork, cod"

### Zero shot

In [ ]:
def zero_shot_llama_2(instructions, ingredients):
  # zero shot function
  prompt = instructions + "\n\nIngredients: " + ingredients

  sequences = llama_pipeline(
      prompt,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
  )
  print("Chatbot:", sequences[0]['generated_text'])

In [ ]:
zero_shot_llama_2(instructions_1, ingredients_1)

Chatbot: Make me a recipe with these ingredients:

Ingredients: Chicken, garlic, spinach, peppers, olive oil, salt, pepper, nutmeg, cumin, paprika.

I want a dish that is easy to make and will be a hit with my family. Can you help me?

Answer: Of course! Here is a simple and delicious recipe that incorporates all of the ingredients you mentioned:

Chicken and Veggie Stir-Fry

Ingredients:

* 1 lb boneless, skinless chicken breast, cut into bite-sized pieces
* 3 cloves garlic, minced
* 1 cup fresh spinach, chopped
* 1 red bell pepper, diced
* 1 green bell pepper, diced
* 2 tbsp olive oil
* Salt and pepper, to taste
* 1/2 tsp nutmeg
* 1/2 tsp cumin
* 1/4 tsp paprika

Instructions:

1. Heat the olive oil in a large skillet or wok over medium-high heat.
2. Add the chicken to the skillet and cook until browned and cooked through, about 5-7 minutes. Remove from the skillet and set aside.
3. Add the minced garlic to the skillet and cook for 1-2 minutes, until fragrant.
4. Add the chopped spin

In [ ]:
zero_shot_llama_2(instructions_2, ingredients_1)

Chatbot: Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”.

Ingredients: Chicken, garlic, spinach, peppers, olive oil, salt, pepper, and flour.

Please help!


In [ ]:
zero_shot_llama_2(instructions_3, ingredients_1)

Chatbot: You are an expert chef who is highly skilled at providing cooking instructions. When given a list of ingredients, please:
1. Provide a step by step list of instructions for cooking a meal.
2. The instructions will only require the listed ingredients.
3. The instructions may choose not to use some of the listed ingredients. 
4. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”

Ingredients: Chicken, garlic, spinach, peppers, olive oil, salt, pepper

Please provide a step by step list of instructions for cooking a meal using the given ingredients.


In [ ]:
zero_shot_llama_2(instructions_2, ingredients_2)

Chatbot: Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”.

Ingredients: Flour, sugar, butter, eggs, milk, chocolate chips, oil, whipped cream, frosting, sprinkles

Please help!


In [ ]:
zero_shot_llama_2(instructions_2, ingredients_3)

Chatbot: Give me step-by-step instructions for a recipe with these ingredients. Be as specific as possible. If you can’t think of a possible recipe, say: “you need to go shopping ASAP”.

Ingredients: Broccoli, strawberries, pineapple, JELLO mix, raisins, pork, cod, chicken, onions, garlic, ginger, cinnamon, nutmeg, and eggs.

Please help!


### Few-shot


In [ ]:
import copy

# process the data a bit more, select 10 examples for few-shot
processed_data = rawData[["NER", "directions"]].loc[0:9,:]
processed_data["ingredients_processed"] = processed_data["NER"].transform(lambda x: ", ".join(ast.literal_eval(x)))
processed_data["directions_processed"] = processed_data["directions"].transform(lambda x: " ".join(ast.literal_eval(x)))

In [ ]:
def few_shot_llama_2(instructions, ingredients, sample):
    # few shot code with 3 examples

    few_shot= "Here are some example recipes:"

    ingr_template = "\nIngredients: "

    recipe_template = "\nRecipe: "

    for i in range(3):
      row = sample.loc[i,:]
      few_shot = few_shot + ingr_template + row["ingredients_processed"] + "\n" + recipe_template + row["directions_processed"] + "\n"

    prompt =  few_shot + "\n" + instructions + "\n\nIngredients: " + ingredients

    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    print("Chatbot:", sequences[0]['generated_text'])


In [ ]:
few_shot_llama_2(instructions_1, ingredients_1, processed_data)

Chatbot: Here are some example recipes:
Ingredients: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits

Recipe: In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.

Ingredients: beef, chicken breasts, cream of mushroom soup, sour cream

Recipe: Place chipped beef on bottom of baking dish. Place chicken on top of beef. Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.

Ingredients: frozen corn, cream cheese, butter, garlic powder, salt, pepper

Recipe: In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.

Make me a recipe with these

In [ ]:
few_shot_llama_2(instructions_2, ingredients_1, processed_data)

Chatbot: Here are some example recipes:
Ingredients: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits

Recipe: In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.

Ingredients: beef, chicken breasts, cream of mushroom soup, sour cream

Recipe: Place chipped beef on bottom of baking dish. Place chicken on top of beef. Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.

Ingredients: frozen corn, cream cheese, butter, garlic powder, salt, pepper

Recipe: In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.

Give me step-by-step instru

In [ ]:
few_shot_llama_2(instructions_3, ingredients_1, processed_data)

Chatbot: Here are some example recipes:
Ingredients: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits

Recipe: In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.

Ingredients: beef, chicken breasts, cream of mushroom soup, sour cream

Recipe: Place chipped beef on bottom of baking dish. Place chicken on top of beef. Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.

Ingredients: frozen corn, cream cheese, butter, garlic powder, salt, pepper

Recipe: In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.

You are an expert chef who 

In [ ]:
few_shot_llama_2(instructions_2, ingredients_2, processed_data)

Chatbot: Here are some example recipes:
Ingredients: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits

Recipe: In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.

Ingredients: beef, chicken breasts, cream of mushroom soup, sour cream

Recipe: Place chipped beef on bottom of baking dish. Place chicken on top of beef. Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.

Ingredients: frozen corn, cream cheese, butter, garlic powder, salt, pepper

Recipe: In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.

Give me step-by-step instru

In [ ]:
few_shot_llama_2(instructions_2, ingredients_3, processed_data)

Chatbot: Here are some example recipes:
Ingredients: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits

Recipe: In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons, drop and shape into 30 clusters on wax paper. Let stand until firm, about 30 minutes.

Ingredients: beef, chicken breasts, cream of mushroom soup, sour cream

Recipe: Place chipped beef on bottom of baking dish. Place chicken on top of beef. Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.

Ingredients: frozen corn, cream cheese, butter, garlic powder, salt, pepper

Recipe: In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.

Give me step-by-step instru